In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r'D:\IT6040\project_excel_themes\gd\csv\mf.csv')


In [3]:
to_drop =['a','b','c','d','e']
data.drop(to_drop, inplace=True, axis=1)

In [4]:
data.dropna(axis=1, how='all')#remove all NaN value

,comments
0,"Unfortunately, I have witnessed a very ""boys c..."
1,The main area I notice a lack of gender divers...
2,I believe that there are efforts from the lowe...
3,it's not just about the gender but also divers...
4,We need to employ/retain or promote women at t...
...,...
159,My perception is the issues are heaviest at th...
160,I'm too new at XXX to comment.
161,There has been an absence of *competent* women...
162,"There are a lot of female employees, but most ..."


In [5]:
data.apply(lambda x: x.astype(str).str.lower())#change all characters in lowercase

,comments
0,"unfortunately, i have witnessed a very ""boys c..."
1,the main area i notice a lack of gender divers...
2,i believe that there are efforts from the lowe...
3,it's not just about the gender but also divers...
4,we need to employ/retain or promote women at t...
...,...
159,my perception is the issues are heaviest at th...
160,i'm too new at xxx to comment.
161,there has been an absence of *competent* women...
162,"there are a lot of female employees, but most ..."


In [6]:

from langdetect import detect
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

D:\anaconda\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [7]:
print(data)

                                              comments
0    Unfortunately, I have witnessed a very "boys c...
1    The main area I notice a lack of gender divers...
2    I believe that there are efforts from the lowe...
3    it's not just about the gender but also divers...
4    We need to employ/retain or promote women at t...
..                                                 ...
159  My perception is the issues are heaviest at th...
160                     I'm too new at XXX to comment.
161  There has been an absence of *competent* women...
162  There are a lot of female employees, but most ...
163                                                NaN

[164 rows x 1 columns]


In [8]:
data = data.dropna().reset_index(drop=True)

In [9]:
data

,comments
0,"Unfortunately, I have witnessed a very ""boys c..."
1,The main area I notice a lack of gender divers...
2,I believe that there are efforts from the lowe...
3,it's not just about the gender but also divers...
4,We need to employ/retain or promote women at t...
...,...
158,I have seen so many strong director level wome...
159,My perception is the issues are heaviest at th...
160,I'm too new at XXX to comment.
161,There has been an absence of *competent* women...


In [10]:
#tokenization for the first 3 sentences of the first article
from nltk.tokenize import sent_tokenize

In [11]:

data['sentences'] = data.comments.progress_map(sent_tokenize)
data['sentences'].head(1).tolist()[0][:3] # Print the first 3 sentences of the 1st article


['Unfortunately, I have witnessed a very "boys club" atmosphere at XXX']

In [12]:
from nltk.tokenize import word_tokenize

data['tokens_sentences'] = data['sentences'].progress_map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
print(data['tokens_sentences'].head(1).tolist()[0][:3])


[['Unfortunately', ',', 'I', 'have', 'witnessed', 'a', 'very', '``', 'boys', 'club', "''", 'atmosphere', 'at', 'XXX']]


In [13]:
#Lemmatizing with POS tagging
from nltk import pos_tag
data['POS_tokens'] = data['tokens_sentences'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(data['POS_tokens'].head(1).tolist()[0][:3])



[[('Unfortunately', 'RB'), (',', ','), ('I', 'PRP'), ('have', 'VBP'), ('witnessed', 'VBN'), ('a', 'DT'), ('very', 'RB'), ('``', '``'), ('boys', 'NNS'), ('club', 'NN'), ("''", "''"), ('atmosphere', 'RB'), ('at', 'IN'), ('XXX', 'NN')]]


In [14]:
# https://stackoverflow.com/a/15590384
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [15]:
# Lemmatizing each word with its POS tag, in each sentence
data['tokens_sentences_lemmatized'] = data['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

In [16]:
data['tokens_sentences_lemmatized'].head(1).tolist()[0][:3]

[['Unfortunately',
  ',',
  'I',
  'have',
  'witness',
  'a',
  'very',
  '``',
  'boy',
  'club',
  "''",
  'atmosphere',
  'at',
  'XXX']]

In [17]:

from nltk.corpus import stopwords
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
stopwords_other = ['xxx','one', 'mr', 'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something']
my_stopwords = stopwords.words('English') + stopwords_verbs + stopwords_other

In [18]:

from itertools import chain # to flatten list of sentences of tokens into list of tokens

In [19]:

data['tokens'] = data['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
data['tokens'] = data['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

In [20]:

data['tokens'].head(1).tolist()[0][:30]

['unfortunately', 'witness', 'boy', 'club', 'atmosphere']

In [21]:
#LDA data preparation 
from gensim.models import Phrases

In [22]:

tokens = data['tokens'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])

In [23]:
#prepare objects for LDA gensim implementation
from gensim import corpora
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

In [24]:
#Running LDA
from gensim import models
import numpy as np

In [31]:
np.random.seed(123456)
num_topics = 50
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))


Wall time: 339 ms


In [32]:
#SOME OUTCOME OF LDA
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.081*"team" + 0.040*"focus" + 0.020*"equal" + 0.020*"every" + 0.020*"else" + 0.020*"talk" + 0.020*"commitment" + 0.020*"right" + 0.020*"slt" + 0.020*"lack" + 0.020*"way" + 0.020*"technical" + 0.020*"time" + 0.020*"old" + 0.020*"level" + 0.020*"gender" + 0.020*"address" + 0.020*"value" + 0.020*"often" + 0.020*"put"

1: 0.042*"company" + 0.039*"senior_leadership" + 0.030*"gender_diversity" + 0.029*"position" + 0.029*"think" + 0.029*"diversity" + 0.029*"job" + 0.027*"benefit" + 0.021*"level" + 0.020*"start" + 0.020*"executive_level" + 0.020*"director" + 0.020*"believe" + 0.020*"great" + 0.020*"good" + 0.019*"move" + 0.018*"head" + 0.018*"definitely" + 0.018*"felt" + 0.018*"pretty"

2: 0.171*"gender" + 0.119*"experience" + 0.112*"performance" + 0.088*"leadership" + 0.060*"role" + 0.060*"less" + 0.060*"discrimination" + 0.060*"rather" + 0.060*"company" + 0.060*"skill" + 0.001*"behavior" + 0.001*"strong" + 0.001*"executive_director" + 0.001*"often" + 0.001*"little" + 0.001*"give" + 0.001

22: 0.049*"comment" + 0.044*"gender_diversity" + 0.033*"leadership" + 0.032*"diversity" + 0.030*"think" + 0.030*"put" + 0.024*"executive" + 0.022*"general" + 0.021*"work" + 0.021*"feel" + 0.020*"advance" + 0.020*"men" + 0.020*"top" + 0.020*"next" + 0.020*"career" + 0.020*"actually" + 0.020*"always" + 0.015*"especially" + 0.015*"due" + 0.011*"level"

23: 0.089*"pretty" + 0.060*"feel" + 0.059*"day" + 0.030*"equal" + 0.030*"problem" + 0.030*"advance" + 0.030*"boys_club" + 0.030*"sure" + 0.030*"personal" + 0.030*"team" + 0.030*"hard" + 0.030*"gender_diversity" + 0.030*"level" + 0.030*"however" + 0.030*"leader" + 0.030*"good" + 0.030*"senior_leadership" + 0.030*"way" + 0.030*"look" + 0.030*"often"

24: 0.061*"men" + 0.061*"year" + 0.058*"promote" + 0.058*"executive_leadership" + 0.058*"enough_woman" + 0.058*"else" + 0.053*"male" + 0.051*"senior" + 0.038*"leadership" + 0.031*"overall" + 0.031*"leader" + 0.031*"position" + 0.031*"lack" + 0.031*"initiative" + 0.031*"action" + 0.031*"sure" + 0.

44: 0.113*"leadership" + 0.081*"example" + 0.079*"gender_diversity" + 0.053*"executive_leadership" + 0.047*"feel" + 0.031*"clear" + 0.031*"elt" + 0.031*"unconscious_bias" + 0.031*"number" + 0.031*"year" + 0.031*"concern" + 0.031*"significant" + 0.031*"improve" + 0.031*"departure" + 0.031*"level" + 0.031*"discuss" + 0.031*"instead" + 0.031*"little" + 0.031*"initiative" + 0.031*"hr"

45: 0.067*"good" + 0.067*"matter" + 0.067*"slt" + 0.066*"leadership" + 0.065*"hear" + 0.065*"develop" + 0.065*"skill" + 0.065*"many" + 0.065*"grow" + 0.065*"promote" + 0.064*"lack" + 0.064*"position" + 0.006*"senior_leadership_role" + 0.006*"shift" + 0.004*"much" + 0.004*"ca" + 0.004*"issue" + 0.004*"critical" + 0.003*"diversity_issue" + 0.003*"team"

46: 0.069*"leadership" + 0.050*"change" + 0.041*"men" + 0.035*"nothing" + 0.035*"manager" + 0.035*"point" + 0.035*"part" + 0.030*"employee" + 0.027*"group" + 0.027*"much" + 0.026*"hard" + 0.026*"priority" + 0.023*"opportunity" + 0.020*"company" + 0.020*"work" +

In [33]:
print(data.comments.loc[0][:200])

Unfortunately, I have witnessed a very "boys club" atmosphere at XXX


In [34]:
lda_model[corpus[4]]

[(43, 0.96370363)]

In [35]:
#predicting topics
document = '''I see a lot of Women in low to mid level positions while I see more men in management and upper management.  '''
tokens = word_tokenize(document)
topics = lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(tokens)]], columns=['topic #', 'weight', 'words in topic'])

,topic #,weight,words in topic
0,6,0.6,"0.176*""level"" + 0.127*""look"" + 0.117*""leadersh..."
1,24,0.3,"0.061*""men"" + 0.061*""year"" + 0.058*""promote"" +..."
